# Preprocessed Dataset : PairBreakingBadDataset

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
sys.path.append("../src/geotransformer/experiments/lomatch")
sys.path.append("../")

import torch
print(torch.cuda.is_available())
from util.data import PairBreakingBadDataset
from config import make_cfg
from dataset import train_valid_data_loader

torch.multiprocessing.set_start_method('spawn')

import jhutil
import yaml
import torch
torch.manual_seed(1234)

True


In [2]:
%matplotlib widget

In [3]:
from multi_part_assembly.datasets.geometry_data import build_geometry_dataset, build_geometry_dataloader
from multi_part_assembly.datasets.geometry_data import GeometryPartDataset as BreakingBadDataset

import jhutil
import torch
from torch.utils.data import Dataset, DataLoader
import torch
from jhutil import show_meshes, show_point_clouds

In [4]:
# cfg = make_cfg()
# train_loader, val_loader, neighbor_limits = train_valid_data_loader(cfg, False)




NameError: name 'cfg' is not defined

In [ ]:

train_data, val_data = build_geometry_dataset(cfg)


In [ ]:
dir2idx = {
    "/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/mode_5" : 0 ,           # 3
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_45" : 1 ,           # 11
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_38" : 2 ,           # 10
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/mode_16" : 3 ,           # 11
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_13" : 4 ,           # 5
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/mode_11" : 5 ,           # 6
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_35" : 6 ,           # 13
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_42" : 7 ,           # 11
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_7" : 8 ,           # 5
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_48" : 9 ,           # 12
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/mode_2" : 10,           # 3
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/mode_8" : 11,           # 6
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_3" : 12,           # 7
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_9" : 13,           # 9
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_31" : 14,           # 4
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/mode_6" : 15,           # 5
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_74" : 16,           # 3
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_22" : 17,           # 3
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_55" : 18,           # 6
"/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_28" : 19,           # 3
}

In [ ]:

n = 6
dir_loc = "/data/wlsgur4011/DataCollection/BreakingBad/data_split/artifact/73400_sf/fractured_55"

result_path = "/data/wlsgur4011/part_assembly/result/artifact/73400_sf/fractured_55/"


data = train_data[dir2idx[dir_loc]]

In [ ]:
show_meshes(dir_loc)

In [ ]:
broken_pcs = data['broken_pcs']
quat = data['quat']
trans = data['trans']

In [ ]:
from jhutil import quat_trans_transform, matrix_transform

## GT

In [ ]:
broken_pcs_registered = []

for bp, q, t in zip(broken_pcs, quat, trans):
    bp = torch.Tensor(bp)
    q = torch.Tensor(q)
    t = torch.Tensor(t)
    bp_registed = quat_trans_transform(q, t, bp)
    broken_pcs_registered.append(bp_registed)

In [ ]:
[torch.Tensor(pcs) for pcs in broken_pcs_registered]

In [ ]:
# show_point_clouds([broken_pcs_registered[0], broken_pcs_registered[4]])
show_point_clouds(broken_pcs_registered)

## Estimated

In [ ]:
import numpy as np


In [ ]:

from sghr.Laplacian_TS import pair2globalT_cycle

def onegraph(dirpath, n):
    # whether we use the ground truth overlap matrix
    # pairwise registration
    Ts = np.zeros([n,n,4,4])
    irs = np.zeros([n,n])
    weights = np.ones([n,n])
    
    for i in range(n):
        for j in range(n):
            filename = f"piece_{i}:piece_{j}.npz"
            fullpath = os.path.join(dirpath, filename)
            if not os.path.isfile(fullpath):
                continue
            if i != 0 and j != 4:
                continue
            Tij, ir, n_matches = load_data(fullpath)
            
            # guarantee meaningful rotation matrix
            if np.linalg.det(Tij[0:3,0:3])<0:
                Tij[0:2] = Tij[[1,0]]
            # we use ransac's inlier number/100
            irs[i,j], irs[j,i] = ir*n_matches/100, ir*n_matches/100
            Ts[j,i] = Tij
            Ts[i,j] = np.linalg.inv(Tij)
    # conduct the global transformation syn
    Tglobals,weights_out = pair2globalT_cycle(weights*(irs ** 3), Ts, 100)
    return Tglobals

In [ ]:
def load_data(fullpath):
	# 1. load data from using i, j
    data = dict(np.load(fullpath))
    Tij = data["estimated_transform"]
    n_matches = len(data["ref_corr_points"])
	# 2. inlier ratio logic
    ref_corr_points = torch.Tensor(data["ref_corr_points"])
    src_corr_points = torch.Tensor(data["src_corr_points"])
    estimated_transform = torch.Tensor(data["estimated_transform"])
    src_transformed = jhutil.matrix_transform(estimated_transform, src_corr_points)
    
    distances, _ = jhutil.knn(ref_corr_points, src_transformed)
    count = torch.sum(distances < 0.01)
    ir = count / n_matches
    return Tij, ir, n_matches

In [ ]:
Tglobals = onegraph(result_path, n)
registered_list = []

for bp, t in zip(broken_pcs, Tglobals):
    bp = torch.Tensor(bp)
    q = torch.Tensor(q)
    t = torch.Tensor(t)
    bp_registed = matrix_transform(t, bp)
    registered_list.append(bp_registed)
    
show_point_clouds(registered_list)

In [ ]:
i = 0; j=4
filename = f"piece_{i}:piece_{j}.npz"
fullpath = os.path.join(result_path, filename)
data = dict(np.load(fullpath))

In [ ]:

import jhutil;jhutil.jhprint(7777, data)

In [ ]:
tranfroms

In [ ]:
broken_pcs_registered_estimated = []
for bp, t in zip(broken_pcs, tranfroms):
    bp = torch.Tensor(bp)
    bp_registed = matrix_transform(t, bp)
    broken_pcs_regxistered_estimated.append(bp_registed)

In [ ]:
show_point_clouds(broken_pcs_registered_estimated[:2])